In [1]:
%matplotlib qt
import os
import shutil
from pathlib import Path
import pandas as pd
import pickle
import mne

In [ ]:
def get_sorted_mapping(
    site_dir, 
    name_filter=None, 
    suffix_filter=None, 
    exclude_substr=None, 
    key_length=5, 
    file_level="nested",
    id_map_df=None,
):
    if isinstance(name_filter, str):
        name_filter = [name_filter]

    fnames = []
    if file_level == "nested":
        for subject_folder in site_dir.iterdir():
            if subject_folder.is_dir():
                for fname in subject_folder.iterdir():
                    if not fname.name.startswith("._"):
                        if name_filter and not any(nf in fname.name for nf in name_filter):
                            continue
                        if exclude_substr and exclude_substr in fname.name:
                            continue
                        if suffix_filter and fname.suffix != suffix_filter:
                            continue
                        fnames.append(fname)
    else:
        for fname in site_dir.iterdir():
            if fname.is_file() and not fname.name.startswith("._"):
                if name_filter and not any(nf in fname.name for nf in name_filter):
                    continue
                if exclude_substr and exclude_substr in fname.name:
                    continue
                if suffix_filter and fname.suffix != suffix_filter:
                    continue
                fnames.append(fname)

    keys = [fname.stem[:key_length] for fname in fnames]

    if id_map_df is not None:
        id_map = dict(zip(id_map_df["old_id"].astype(str), id_map_df["new_id"].astype(str)))
        keys = [id_map.get(k, k) for k in keys]

    mapping = dict(sorted(zip(keys, fnames)))
    return mapping

if __name__ == "__main__":

    main_dir = Path("/Volumes/Extreme_SSD/payam_data/Tinnorm")
    df = pd.read_csv("/Users/payamsadeghishabestari/Downloads/ant_to_tide (3).csv")
    df = pd.read_csv("/Users/payamsadeghishabestari/Downloads/ant_to_tide (3).csv")
    df.loc[len(df)] = [280, "dsno", 70053, "rest"]
    df.loc[len(df)] = [281, "mkbb", 70054, "rest"]
    df.loc[len(df)] = [282, "sgwy", 70055, "rest"]

    mappings = {
        "dublin": get_sorted_mapping(
            main_dir / "dublin",
            name_filter="open",
            key_length=5,
            file_level="nested",
        ),
        "illinois": get_sorted_mapping(
            main_dir / "illinois",
            name_filter="ses-1_rest",
            key_length=5,
            suffix_filter=".cdt",
            file_level="nested",
        ),
        "regensburg": get_sorted_mapping(
            main_dir / "regensburg",
            exclude_substr="ses",
            suffix_filter=".vhdr",
            key_length=5,
            file_level="nested",
        ),
        "tuebingen": get_sorted_mapping(
            main_dir / "tuebingen",
            suffix_filter=".vhdr",
            key_length=5,
            file_level="nested",
        ),
        "zuerich": get_sorted_mapping(
            main_dir / "zuerich",
            name_filter="rest.vhdr",
            key_length=4,
            file_level="flat",
            id_map_df=df,
        ),
    }

    with open("../material/fname_dict.pkl", "wb") as f:
        pickle.dump(mappings, f)

In [ ]:
raw_list = list()
bids_list = list()
for subject_id in subject_ids:
    for run in runs:
        raw_fname = eegbci.load_data(subject=subject_id, runs=run)[0]
        raw = mne.io.read_raw_edf(raw_fname)
        raw.info["line_freq"] = 50  # specify power line frequency
        raw_list.append(raw)
        bids_path = BIDSPath(
            subject=f"{subject_id:03}",
            session="01",
            task="MotorImagery",
            run=f"{run_map[run]:02}",
            root=bids_root,
        )
        bids_list.append(bids_path)

daysback_min, daysback_max = get_anonymization_daysback(raw_list)

for raw, bids_path in zip(raw_list, bids_list):
    write_raw_bids(
        raw, bids_path, anonymize=dict(daysback=daysback_min + 2117), overwrite=True
    )

In [ ]:
with open("../material/fname_dict.pkl", "rb") as f:
    mappings = pickle.load(f)

def read_vhdr_input_fname(fname):
    """
    Checks .vhdr and .vmrk data to have same names, otherwise fix them.
    """
    try:
        raw = mne.io.read_raw(fname)
    except:
        with open(fname, "r") as file:
            lines = file.readlines()
        
        lines[5] = f'DataFile={fname.stem}.eeg\n'
        lines[6] = f'MarkerFile={fname.stem}.vmrk\n'

        with open(fname, "w") as file:
            file.writelines(lines)
        with open(f"{fname.with_suffix('')}.vmrk", "r") as file:
            lines = file.readlines()
        lines[4] = f'DataFile={fname.stem}.eeg\n'
        with open(f"{fname.with_suffix('')}.vmrk", "w") as file:
            file.writelines(lines)
        raw = mne.io.read_raw(fname)
    return raw

In [56]:
from mne_bids import (
                        BIDSPath,
                        write_raw_bids,
                    )
from autoreject import AutoReject

In [ ]:
sites = ["dublin", "illinois", "regensburg", "tuebingen", "zuerich"]


montage = mne.channels.make_standard_montage("easycap-M1")


for site in sites:
    for subject_id, fname in mappings[site].items():

        if site == "dublin":
            raw = mne.io.read_raw(fname, exclude=("EXG"))

        elif site == "illinois":
            dpo_file = fname.with_suffix('.cdt.dpo')
            dpa_file = fname.with_suffix('.cdt.dpa')
            if dpo_file.exists():
                dpo_file.rename(dpa_file)

            raw = mne.io.read_raw(fname)
            raw.drop_channels(["F11", "F12", "FT11", "FT12", "M1", "M2", "Cb1", "Cb2"])
            transform = mne.read_trans("../material/Illinois-trans.fif")
            raw.info["dev_head_t"] = transform

        elif site in ["regensburg", "tuebingen"]:
            raw = read_vhdr_input_fname(fname)

        elif site == "zuerich":
            ch_types = {
                "O1": "eog",
                "O2": "eog",
                "PO7": "eog",
                "PO8": "eog",
                "Pulse": "ecg",
                "Resp": "ecg",
                "Audio": "stim"
            }
            raw = read_vhdr_input_fname(fname)
            raw.set_channel_types(ch_types)

            ## take only eyes open part


        ## create bids path
        bids_kwargs = {
            "subject": subject_id,
            "session": "01",
            "task" :"rest",
            "acquisition": site,
            "root": "../material",

                }

        bids_path_0 = BIDSPath(processing="0", **bids_kwargs)
        bids_path_1 = BIDSPath(processing="1", **bids_kwargs)
        bids_path_2 = BIDSPath(processing="2", **bids_kwargs)
        bids_path_3 = BIDSPath(processing="3", **bids_kwargs)


        ## orig
        raw.pick(["eeg"])
        raw.set_montage(montage)
        raw.load_data()
        raw.resample(sfreq=250)
        
        ## probably save orig raw here
        


        ## preproc 1
        raw.filter(1, 40)
        raw.set_eeg_reference("average", projection=False) # add notch filter

        epochs = mne.make_fixed_length_epochs(raw, duration=10)


        ## preproc 2
        ## epoch it and run 


        ## preproc 3
        ## add ICA




Extracting EDF parameters from /Volumes/Extreme_SSD/payam_data/Tinnorm/dublin/OneDrive_33_11-08-2025/20036_ses-1_rest_open.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1228799  =      0.000 ...   300.000 secs...


<RawEDF | 20036_ses-1_rest_open.bdf, 64 x 1228800 (300.0 s), ~600.1 MiB, data loaded>